In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))

In [11]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [12]:
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [19]:
block_size = 3
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * block_size # list of 0s of blocksize
    for ch in w + '.':
        ix = stoi[ch] # character index
        X.append(context)
        Y.append(ix) #
        print(''.join(itos[i] for i in context), '--->',itos[ix])
        context = context[1:] + [ix] # Take the first 2 items i context and add the ix at the end of the list

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [20]:
X.shape,X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [21]:
C = torch.randn((27,2))

In [22]:
C

tensor([[-1.0451,  0.2271],
        [-0.4246,  0.5392],
        [-0.5737, -0.8957],
        [ 1.4986, -0.5988],
        [-1.3220,  0.0404],
        [-0.2435, -0.5964],
        [ 0.6180, -0.9611],
        [ 0.6871, -1.3060],
        [ 1.3025, -0.3111],
        [-0.4725,  0.9513],
        [ 1.2088,  0.0408],
        [-0.1670,  1.7796],
        [ 0.1698,  1.0846],
        [-2.4542,  1.6683],
        [-0.1402, -0.4956],
        [-0.3065,  0.7363],
        [ 0.5563, -0.1821],
        [-0.1230,  0.1824],
        [-0.1040,  0.9632],
        [ 0.5984,  0.0641],
        [ 1.0266,  0.6260],
        [ 0.6885,  0.8778],
        [-0.7261,  1.4174],
        [ 2.7825, -2.3553],
        [ 0.0887,  0.1935],
        [-1.1268,  0.3802],
        [ 0.6110,  1.1021]])

In [23]:
C[5]

tensor([-0.2435, -0.5964])

In [27]:
F.one_hot(torch.tensor(5),num_classes=27).float() @ C

tensor([-0.2435, -0.5964])

In [29]:
C[torch.tensor([3,4,5])]

tensor([[ 1.4986, -0.5988],
        [-1.3220,  0.0404],
        [-0.2435, -0.5964]])

In [31]:
C[X].shape

torch.Size([32, 3, 2])

In [32]:
X[13,2]

tensor(1)

In [33]:
C[X][13,2]

tensor([-0.4246,  0.5392])

In [34]:
C[1]

tensor([-0.4246,  0.5392])

In [35]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [38]:
# Hidden layer
W1 = torch.randn((6,100)) # input will be emb.shape(x,y,z) ---> y * z, The number of neurons we decide (hyperparameter)
b1 = torch.randn(100)

In [39]:
emb @ W1 + b1 # does not work cause emb.shape(32,3,2) while W1 is (6, 100)
              # 32, 3, 2 x
              #  6, 100

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [41]:
logits = emb.view(-1,6) @ W1  + b1

Multiple ways to change the shape of the embedding

In [46]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]],1) # grab all examples, the [0,1,2] index, all indexes in 3rd dimension

tensor([[-1.0451,  0.2271, -1.0451,  0.2271, -1.0451,  0.2271],
        [-1.0451,  0.2271, -1.0451,  0.2271, -0.2435, -0.5964],
        [-1.0451,  0.2271, -0.2435, -0.5964, -2.4542,  1.6683],
        [-0.2435, -0.5964, -2.4542,  1.6683, -2.4542,  1.6683],
        [-2.4542,  1.6683, -2.4542,  1.6683, -0.4246,  0.5392],
        [-1.0451,  0.2271, -1.0451,  0.2271, -1.0451,  0.2271],
        [-1.0451,  0.2271, -1.0451,  0.2271, -0.3065,  0.7363],
        [-1.0451,  0.2271, -0.3065,  0.7363,  0.1698,  1.0846],
        [-0.3065,  0.7363,  0.1698,  1.0846, -0.4725,  0.9513],
        [ 0.1698,  1.0846, -0.4725,  0.9513, -0.7261,  1.4174],
        [-0.4725,  0.9513, -0.7261,  1.4174, -0.4725,  0.9513],
        [-0.7261,  1.4174, -0.4725,  0.9513, -0.4246,  0.5392],
        [-1.0451,  0.2271, -1.0451,  0.2271, -1.0451,  0.2271],
        [-1.0451,  0.2271, -1.0451,  0.2271, -0.4246,  0.5392],
        [-1.0451,  0.2271, -0.4246,  0.5392, -0.7261,  1.4174],
        [-0.4246,  0.5392, -0.7261,  1.4

In [47]:
view_opt = emb.view(-1,6)
cat_opt = torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]],1)

In [49]:
torch.equal(view_opt,cat_opt)

True

Not good because if we want to change the block size we will haveto manually do it in the code

Instead we should do torch.unbind https://pytorch.org/docs/stable/generated/torch.unbind.html
which removes a tensor dimension

Returns a tuple of all slices along a given dimension, already without it.

In [ ]:
unbind_opt = torch.unbind(emb,1) # Same as the list in cat options

In [56]:
unbind_opt

(tensor([[-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.2435, -0.5964],
         [-2.4542,  1.6683],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.3065,  0.7363],
         [ 0.1698,  1.0846],
         [-0.4725,  0.9513],
         [-0.7261,  1.4174],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.4246,  0.5392],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.4725,  0.9513],
         [ 0.5984,  0.0641],
         [-0.4246,  0.5392],
         [-0.5737, -0.8957],
         [-0.2435, -0.5964],
         [ 0.1698,  1.0846],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [ 0.5984,  0.0641],
         [-0.3065,  0.7363],
         [ 0.5563, -0.1821],
         [ 1.3025, -0.3111]]),
 tensor([[-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0

In [60]:
([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]])

[tensor([[-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.2435, -0.5964],
         [-2.4542,  1.6683],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.3065,  0.7363],
         [ 0.1698,  1.0846],
         [-0.4725,  0.9513],
         [-0.7261,  1.4174],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.4246,  0.5392],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0.4725,  0.9513],
         [ 0.5984,  0.0641],
         [-0.4246,  0.5392],
         [-0.5737, -0.8957],
         [-0.2435, -0.5964],
         [ 0.1698,  1.0846],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [ 0.5984,  0.0641],
         [-0.3065,  0.7363],
         [ 0.5563, -0.1821],
         [ 1.3025, -0.3111]]),
 tensor([[-1.0451,  0.2271],
         [-1.0451,  0.2271],
         [-0

In [61]:
torch.cat(unbind_opt,1)

tensor([[-1.0451,  0.2271, -1.0451,  0.2271, -1.0451,  0.2271],
        [-1.0451,  0.2271, -1.0451,  0.2271, -0.2435, -0.5964],
        [-1.0451,  0.2271, -0.2435, -0.5964, -2.4542,  1.6683],
        [-0.2435, -0.5964, -2.4542,  1.6683, -2.4542,  1.6683],
        [-2.4542,  1.6683, -2.4542,  1.6683, -0.4246,  0.5392],
        [-1.0451,  0.2271, -1.0451,  0.2271, -1.0451,  0.2271],
        [-1.0451,  0.2271, -1.0451,  0.2271, -0.3065,  0.7363],
        [-1.0451,  0.2271, -0.3065,  0.7363,  0.1698,  1.0846],
        [-0.3065,  0.7363,  0.1698,  1.0846, -0.4725,  0.9513],
        [ 0.1698,  1.0846, -0.4725,  0.9513, -0.7261,  1.4174],
        [-0.4725,  0.9513, -0.7261,  1.4174, -0.4725,  0.9513],
        [-0.7261,  1.4174, -0.4725,  0.9513, -0.4246,  0.5392],
        [-1.0451,  0.2271, -1.0451,  0.2271, -1.0451,  0.2271],
        [-1.0451,  0.2271, -1.0451,  0.2271, -0.4246,  0.5392],
        [-1.0451,  0.2271, -0.4246,  0.5392, -0.7261,  1.4174],
        [-0.4246,  0.5392, -0.7261,  1.4

Even better way

# Next time